In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
from PIL import Image

In [ ]:
# Define Parameters
NUM_CLASS = 5
IMG_SIZE = 224

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

In [ ]:
# download from google drive or upload file by yourself
# !pip install --upgrade gdown
# !gdown --fuzzy '1jXEzNJr7iYJzhzl7695_0scTqJB0_5L7' --output retina-dataset.zip

In [ ]:
# unzip dataset file
# !unzip -q retina-dataset.zip

#### Data Visualization

In [ ]:
df = pd.read_csv("kaggle_retina/train.csv")
df.head()

In [ ]:
paths = glob('kaggle_retina/train/*.png')
paths_test = sorted(glob('kaggle_retina/test/*.png'))
len(paths), len(paths_test)

In [ ]:
idx = np.random.randint(0, len(df))
row = df.iloc[idx]
filename = row["id_code"]
path = f"kaggle_retina/train/{filename}.png"
img = cv2.imread(path)[:,:,::-1]
plt.imshow(img)
print(path)
print(f'label: {row["diagnosis"]}')

In [ ]:
all_cls = df['diagnosis'].tolist()
print(all_cls[:3])

In [ ]:
# Class distribution
cls, count = np.unique(all_cls, return_counts=True)
plt.bar(cls, count)

In [ ]:
print(list(zip(cls, count)))

In [ ]:
weights = [1, 2, 1, 5, 3]
class_weights = torch.FloatTensor(weights).to(device)

#### [TODO] Dataset, DataLoader

Dataset & Data augumentation

#### Dataset & DataLoader


*   [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset)



*   Must implement

    1.   **\_\_len\_\_()**: size of dataset
    2.   **\_\_getitem\_\_()**: load a data sample for a given index

*   __getitem__() steps:
    1. Get image path
    2. Read image, resize
    3. Real image label
    4. convert image to tesnor and rescale to 0~1


ref: [Datasets & DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)


In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
from torchvision import models, transforms
# from torchvision.models import googlenet, GoogLeNet_Weights
# from torchvision.models import densenet161, DenseNet161_Weights
from torchvision.models import resnext50_32x4d, ResNeXt50_32X4D_Weights

In [ ]:
transform = ResNeXt50_32X4D_Weights.DEFAULT.transforms()
transform

In [ ]:
class RetinaDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform, aug):
        self.df = df
        self.transform = transform
        self.seq = iaa.Sequential([
            iaa.Fliplr(0.5), # 50% horizontal flip
            iaa.Flipud(0.5), # 50% vertical flip
            iaa.Affine(
                rotate=(-10, 10), # random rotate -10 ~ +10 degree
                shear=(-16,16), # random shear -16 ~ +16 degree
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)} # scale x, y: 80%~120%
            ),
        ])
        self.aug = aug


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row["id_code"]
        path = f"kaggle_retina/train/{filename}.png"
        img = Image.open(path)
        img_np = np.array(img)
        if self.aug:
          img_np = self.seq.augment_image(img_np)
        img = Image.fromarray(img_np)

        img = self.transform(img)



        label = row["diagnosis"]
        label = torch.tensor(label, dtype=torch.long)

        return img, label

In [ ]:
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, img_paths, df, Transform):
#         self.img_paths = img_paths
#         # self.img_size = img_size
#         self.df = df
#         self.transform = Transform

#         # self.seq = seq = iaa.Sequential([
#         #     iaa.Fliplr(0.5), # 50% horizontal flip
#         #     iaa.Flipud(0.5), # 50% vertical flip
#         #     iaa.Affine(
#         #         rotate=(-10, 10), # random rotate -10 ~ +10 degree
#         #         shear=(-16,16), # random shear -16 ~ +16 degree
#         #         scale={"x": (0.8, 1.2), "y": (0.8, 1.2)} # scale x, y: 80%~120%
#         #     ),
#         # ])
#         # self.aug = aug
#     def __len__(self):
#         """number of samples"""
#         return len(self.img_paths)

#     def __getitem__(self, idx):
#         """read 1 sample"""
#         # Read img
#         row = df.iloc[idx]
#         filename = row["id_code"]
#         path = f"kaggle_retina/train/{filename}.png" # get img path
#         img = cv2.imread(path) # read img
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
#         # img = cv2.resize(img, (self.img_size, self.img_size)) # Resize iamge
#         # if self.aug:
#         #     img = self.seq.augment_image(img)
#         img = img / 255. # 0~255 -> 0.~1.
#         img = self.transform(img)
#         # Read class index
#         cls_idx = row['diagnosis']

#         # transform img to tensor
#         img = torch.tensor(img, dtype=torch.float)
#         # img = self.transform(img)
#         # channel last -> channel first
#         # np.swapaxis()
#         img = img.permute(2, 0, 1) # (H, W, C) -> (C, H, W)

#         cls_idx = torch.tensor(cls_idx, dtype=torch.long)

#         return img, cls_idx

In [ ]:
# Build training and validation dataset

remove_ratio = 0.5
df = pd.read_csv('kaggle_retina/train.csv')
print(df)
rows_to_remove = df[df['diagnosis'] == 0].sample(frac=remove_ratio, random_state=42).index
df = df.drop(rows_to_remove)
print(df)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1342)
# remove data from class 0
# rows_to_remove = train_df[train_df['diagnosis'] == 0].sample(frac=remove_ratio, random_state=42).index
# print(train_df)
# train_df = train_df.drop(rows_to_remove)
# print(train_df)

train_ds = RetinaDataset(train_df,transform,aug=True)
val_ds = RetinaDataset(val_df,transform,aug=False)

In [ ]:
all_cls = df['diagnosis'].tolist()
cls, count =np.unique(all_cls, return_counts=True)
plt.bar(cls, count)

In [ ]:
# Number of samples
len(train_ds), len(val_ds)

In [ ]:
# Build DataLoaders
# BATCH_SIZE = 50
BATCH_SIZE = 62
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE)

In [ ]:
idx = np.random.randint(0, len(train_ds))
img, label = train_ds[idx]
print(idx, label)
plt.subplot(1, 2, 1)
plt.imshow(img.permute(1, 2, 0))

# Convert back to original value
img_raw = img.numpy().transpose(1, 2, 0) # (3, 256, 256) -> (256, 256, 3)
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img_raw = std * img_raw + mean
img_raw = np.clip(img_raw, 0, 1)
print(img_raw.shape)
plt.subplot(1, 2, 2)
plt.imshow(img_raw)
plt.show()
# img, label = train_ds[0]
# print(img.shape, label)

# img = img.permute(1, 2, 0) # (C, H, W) -> (H, W, C)
# plt.imshow(img)
# plt.show()

In [ ]:
img, label

#### [TODO] Model

In [ ]:
# 繼承nn.Module,初始__init__化定義有哪些層（參數輸入input channel數,output channel數）,forward函數定義從資料進來到出去的步驟。
# class ConvBN(nn.Module):
#     def __init__(self, cin, cout, kernel_size = 3):
#         super().__init__()
#         self.conv = nn.Conv2d(cin, cout, kernel_size, padding='same')
#         self.bn = nn.BatchNorm2d(num_features=cout)
#         self.act = nn.LeakyReLU()
#     def forward(self, x):
#         x = self.conv(x)
#         x = self.bn(x)
#         return self.act(x)

# class CNN(nn.Module):
#     def __init__(self, kernel_size=3):
#         super().__init__()
#         self.encoder = nn.Sequential(
#             ConvBN(3, 16, kernel_size ),
#             ConvBN(16, 16, kernel_size),
#             nn.MaxPool2d(kernel_size=2),

#             ConvBN(16, 32, kernel_size),
#             ConvBN(32, 32, kernel_size),
#             nn.MaxPool2d(kernel_size=2),

#             ConvBN(32, 64,kernel_size),
#             ConvBN(64, 64,kernel_size),
#             nn.MaxPool2d(kernel_size=2),
#         )
#         self.flatten = nn.Flatten() # 輾平後特徵數量為通道＊長＊寬，其中長寬要計算卷幾過程中的改變
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.3),
#             nn.Linear(64*28*28, 256),
#             nn.Dropout(0.3),
#             nn.LeakyReLU(),
#             nn.Linear(256, NUM_CLASS),
#         )
#         # 使用crossentropyloss作為損失函數時，函數損失不參考數值只看正確比率故在線性層不需使用softmax函數
#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.flatten(x)
#         x = self.classifier(x)
#         return x

In [ ]:
Freeze = True

In [ ]:
model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.DEFAULT)
print(model)
# 引用在參數中把使用imagenet預訓練的權重指定給weights
# 同時檢視模型中線性層的名稱（fc）

In [ ]:
# 要替代現性層故要知道輸入（卷積層的輸出）features 數量
num_feature = model.fc.in_features

# 如果要凍結參數
if Freeze:
  for param in model.parameters():
    param.requires_grad = False
# 多層連階層
model.fc = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(num_feature , 128),
            nn.Dropout(0.4),
            nn.LeakyReLU(),
            nn.Linear(128, NUM_CLASS))
# 單連接層
# model.fc = nn.Linear(num_feature, NUM_CLASS)

In [ ]:
inputs = torch.randn(1, 3, 224, 224).to(device)
model = model.to(device)
outputs = model(inputs)
outputs.shape

In [ ]:
# from torchsummary import summary # model summary
# Model summary
# summary(model, (3, 224, 224))

#### [TODO] Training

In [ ]:
# 定義損失函數與激活函數
loss_fn = nn.CrossEntropyLoss(weight= class_weights)
optimizer = torch.optim.AdamW(model.parameters(),
                            lr=1e-4, weight_decay=1)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode.
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
from tqdm.auto import tqdm
EPOCHS = 100

logs = {
    'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
}
erly_stop = 5
best_loss = np.inf
counter = 0
# best_model = model
# np.inf-無限大
for epoch in tqdm(range(EPOCHS)):
    train_loss, train_acc = train(train_loader, model, loss_fn, optimizer)
    val_loss, val_acc = test(val_loader, model, loss_fn)

    print(f'EPOCH: {epoch:04d} \
    train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
    val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ', counter)

    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)
    torch.save(model.state_dict(), "last.pth")
    # chcek improvement
    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        torch.save(model.state_dict(), "best.pth")
    else:
        counter += 1
    if counter >= erly_stop:
        print("Earlystop!")
        break

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH: 0000     train_loss: 1.6102, train_acc: 0.201     val_loss: 1.6054, val_acc: 0.281  0


  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH: 0001     train_loss: 1.6032, train_acc: 0.223     val_loss: 1.6025, val_acc: 0.251  0


...

  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH: 0004     train_loss: 1.5919, train_acc: 0.316     val_loss: 1.5837, val_acc: 0.423  0


  0%|          | 0/22 [00:00<?, ?it/s]

EPOCH: 0005     train_loss: 1.5826, train_acc: 0.339     val_loss: 1.5792, val_acc: 0.453  0


  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
# Plot loss curve
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(logs['train_loss'])
plt.plot(logs['val_loss'])
plt.legend(['train_loss', 'val_loss'])
# plot acc
plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(logs['train_acc'])
plt.plot(logs['val_acc'])
plt.legend(['train_acc', 'val_acc'])
plt.show()

In [ ]:
model.load_state_dict(torch.load('best.pth'))

model.eval().to(device)

####Kaggle Submission

https://www.kaggle.com/competitions/diabetic-retinopathy-classification-3/overview

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row["id_code"]
        path = f"kaggle_retina/test/{filename}.png"
        img = Image.open(path)
        img = self.transform(img)
        return img

df = pd.read_csv('kaggle_retina/submission.csv')
test_ds = TestDataset(df, transform)
test_loader = torch.utils.data.DataLoader(test_ds, BATCH_SIZE)

In [ ]:
# Inference
y_pred = []
with torch.no_grad():
    for x in tqdm(test_loader):
        x = x.to(device)
        pred = model(x)
        y_pred.append(pred.argmax(dim=1))

y_pred = torch.cat(y_pred, dim=0).cpu().numpy()

In [ ]:
df['diagnosis'] = y_pred
df.to_csv('submission.csv', index=None)

Model Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Inference
y_pred = []
with torch.no_grad():
    for x ,y in tqdm(val_loader):
        x = x.to(device)
        pred = model(x)
        y_pred.append(pred.argmax(dim=1))

val_pred = torch.cat(y_pred, dim=0).cpu().numpy()

In [ ]:
y_val = val_df['diagnosis'].values
# print(y_val)
print(y_val)
print(val_pred)

In [ ]:
print(classification_report(y_val, val_pred))
print(confusion_matrix(y_val, val_pred))